##### Using mullti variable for Time Series Forcasting

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.layers import SimpleRNN
import yfinance as yahooFinance
from keras.utils import timeseries_dataset_from_array
import tensorflow as tf
import datetime

##### Data Preprocessing 

In [2]:
df = pd.read_csv('BTC-sep-2023-2024.csv', parse_dates=["Date"])
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2023-09-21 00:00:00+00:00,27129.839844,27152.939453,26389.300781,26567.632812,13371443708,0.0,0.0
1,2023-09-22 00:00:00+00:00,26564.056641,26726.078125,26495.533203,26579.568359,10578746709,0.0,0.0
2,2023-09-23 00:00:00+00:00,26578.556641,26634.185547,26520.519531,26579.390625,7404700301,0.0,0.0
3,2023-09-24 00:00:00+00:00,26579.373047,26716.058594,26221.050781,26256.826172,8192867686,0.0,0.0
4,2023-09-25 00:00:00+00:00,26253.775391,26421.507812,26011.468750,26298.480469,11997833257,0.0,0.0
...,...,...,...,...,...,...,...,...
360,2024-09-15 00:00:00+00:00,60000.726562,60381.917969,58696.308594,59182.835938,18120960867,0.0,0.0
361,2024-09-16 00:00:00+00:00,59185.226562,59205.511719,57501.339844,58192.507812,32032822113,0.0,0.0
362,2024-09-17 00:00:00+00:00,58192.507812,61316.089844,57628.070312,60308.539062,38075570118,0.0,0.0
363,2024-09-18 00:00:00+00:00,60309.000000,61664.066406,59218.253906,61649.679688,40990702891,0.0,0.0


In [3]:
df['Date'] = pd.to_datetime(df['Date']).dt.date

In [4]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

In [5]:
df2 = df.sort_values("Date").set_index("Date")
df2

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-09-21,27129.839844,27152.939453,26389.300781,26567.632812,13371443708,0.0,0.0
2023-09-22,26564.056641,26726.078125,26495.533203,26579.568359,10578746709,0.0,0.0
2023-09-23,26578.556641,26634.185547,26520.519531,26579.390625,7404700301,0.0,0.0
2023-09-24,26579.373047,26716.058594,26221.050781,26256.826172,8192867686,0.0,0.0
2023-09-25,26253.775391,26421.507812,26011.468750,26298.480469,11997833257,0.0,0.0
...,...,...,...,...,...,...,...
2024-09-15,60000.726562,60381.917969,58696.308594,59182.835938,18120960867,0.0,0.0
2024-09-16,59185.226562,59205.511719,57501.339844,58192.507812,32032822113,0.0,0.0
2024-09-17,58192.507812,61316.089844,57628.070312,60308.539062,38075570118,0.0,0.0


In [12]:
df3 = df2.drop(['Dividends', 'Stock Splits', 'Volume'], axis='columns')
df3

,Open,High,Low,Close
Date,,,,
2023-09-21,27129.839844,27152.939453,26389.300781,26567.632812
2023-09-22,26564.056641,26726.078125,26495.533203,26579.568359
2023-09-23,26578.556641,26634.185547,26520.519531,26579.390625
2023-09-24,26579.373047,26716.058594,26221.050781,26256.826172
2023-09-25,26253.775391,26421.507812,26011.468750,26298.480469
...,...,...,...,...
2024-09-15,60000.726562,60381.917969,58696.308594,59182.835938
2024-09-16,59185.226562,59205.511719,57501.339844,58192.507812
2024-09-17,58192.507812,61316.089844,57628.070312,60308.539062


##### Split the data into train, valid and test sets

In [13]:
train_data = df3["2023-09":"2024-7-26"] / 1e6
valid_data = df3["2024-07-27":"2024-08"] / 1e6
test_data = df3["2024-09":] / 1e6

In [14]:
seq_length = 20

In [24]:
def split_inputs_and_targets(mulvar_series, ahead=5,target_col=1):
    return mulvar_series[:, :-ahead], mulvar_series[:, -ahead:,target_col]


ahead_train_ds = tf.keras.utils.timeseries_dataset_from_array(train_data.to_numpy(),
 targets=None,
 sequence_length=seq_length + 5,
 batch_size=32,
 shuffle=True,
 seed=42
).map(split_inputs_and_targets)

ahead_valid_ds = tf.keras.utils.timeseries_dataset_from_array(
 valid_data.to_numpy(),
 targets=None,
 sequence_length=seq_length + 5,
 batch_size=32
).map(split_inputs_and_targets)

In [25]:
tf.random.set_seed(42)

In [26]:
model = keras.Sequential([
 keras.layers.LSTM(32, return_sequences=True, input_shape=[None, 4]),
 keras.layers.LSTM(32, return_sequences=True),
 keras.layers.LSTM(10),
 keras.layers.Dense(5)
])

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, None, 32)          4736      
                                                                 
 lstm_4 (LSTM)               (None, None, 32)          8320      
                                                                 
 lstm_5 (LSTM)               (None, 10)                1720      
                                                                 
 dense_1 (Dense)             (None, 5)                 55        
                                                                 
Total params: 14831 (57.93 KB)
Trainable params: 14831 (57.93 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_mae", patience=50, restore_best_weights=True)
opt = keras.optimizers.SGD(learning_rate=0.02, momentum=0.9)
model.compile(loss=keras.losses.Huber(), optimizer=opt,metrics=["mae"])

In [29]:
history = model.fit(ahead_train_ds, validation_data=ahead_valid_ds,epochs=100,callbacks=[early_stopping_cb])

Epoch 1/100
9/9 [==============================] - 114s 714ms/step - loss: 0.0016 - mae: 0.0522 - val_loss: 0.0010 - val_mae: 0.0446
Epoch 2/100
9/9 [==============================] - 0s 48ms/step - loss: 4.4008e-04 - mae: 0.0251 - val_loss: 1.6022e-04 - val_mae: 0.0153
Epoch 3/100
9/9 [==============================] - 0s 44ms/step - loss: 2.0838e-04 - mae: 0.0168 - val_loss: 5.4731e-05 - val_mae: 0.0080
Epoch 4/100
9/9 [==============================] - 0s 47ms/step - loss: 1.8151e-04 - mae: 0.0153 - val_loss: 1.7878e-05 - val_mae: 0.0052
Epoch 5/100
9/9 [==============================] - 0s 46ms/step - loss: 1.3127e-04 - mae: 0.0139 - val_loss: 3.8109e-05 - val_mae: 0.0074
Epoch 6/100
9/9 [==============================] - 0s 45ms/step - loss: 1.1823e-04 - mae: 0.0137 - val_loss: 5.3688e-05 - val_mae: 0.0094
Epoch 7/100
9/9 [==============================] - 0s 43ms/step - loss: 1.1833e-04 - mae: 0.0138 - val_loss: 4.9100e-05 - val_mae: 0.0093
Epoch 8/100
9/9 [======================

In [30]:
model.save('multivar_deep_v1_1.h5')

c:\Users\TeeFaith\anaconda3\envs\kenny\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [31]:
X = valid_data.to_numpy()[np.newaxis, :seq_length] 
Y_pred = model.predict(X)

1/1 [==============================] - 4s 4s/step


In [32]:
Y_pred.shape

(1, 5)